# San Francisco Rental Prices Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions that can be used for a Panel dashboard.

In [7]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv

In [8]:
# Read the Mapbox API key
load_dotenv()
mapbox_token = os.getenv("MAPBOX_API_KEY")

# Import Data

In [9]:
# Import the CSVs to Pandas DataFrames
file_path = Path(r"C:\Users\Owner\Documents\sfo_neighborhoods_census_data.csv")
sfo_data = pd.read_csv(file_path, index_col="year")

file_path = Path(r"C:\Users\Owner\Documents\neighborhoods_coordinates.csv")
df_neighborhood_locations = pd.read_csv(file_path)

- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

In [21]:
# Define Panel Visualization Functions
def housing_units_per_year():
    
   
   
    sfo_housing_mean_per_year = sfo_data.groupby("year").mean()
    sfo_housing_mean_per_year.reset_index(inplace=True)

    std=sfo_housing_mean_per_year["housing_units"].std()
    mins=sfo_housing_mean_per_year["housing_units"].min()-std
    maxes=sfo_housing_mean_per_year["housing_units"].max()+std

    housing_units_per_year_graph = px.bar(
    sfo_housing_mean_per_year,
    x="year",
    y="housing_units",
    range_y=[mins,maxes],
    labels={"year":"Year","housing_units":"Housing Units"},
    title="Housing Units in San Francisco from 2010 to 2016",
    width=800, 
    height=600
    )
    return housing_units_per_year_graph
    


def average_gross_rent():
    """Average Gross Rent in San Francisco Per Year."""
    

    sfo_housing_mean_per_year = sfo_data.groupby("year").mean()
    sfo_housing_mean_per_year.reset_index(inplace=True)

    average_gross_rent_per_year_graph = px.line(
    sfo_housing_mean_per_year,
    x="year",
    y="gross_rent",
    labels={"year":"Year","gross_rent":"Gross Rent"},
    title="Average Gross Rent in San Francisco",
    width=800, 
    height=600
    )
    return average_gross_rent_per_year_graph

def average_sales_price():
    """Average Sales Price Per Year."""
    
    sfo_data_mean_per_year = sfo_data.groupby("year").mean()
    sfo_data_mean_per_year.reset_index(inplace=True)

    average_sale_price_per_squre_foot_graph = px.line(
    sfo_data_mean_per_year,
    x="year",
    y="gross_rent",
    labels={"year":"Year","gross_rent":"Gross Rent"},
    title="Average Gross Rent in San Francisco",
    width=800, 
    height=600
    )
    return average_sale_price_per_squre_foot_graph


def average_price_by_neighborhood():
    """Average Prices by Neighborhood."""
    
    average_prices_by_neighborhood = sfo_data.groupby(["year","neighborhood"]).mean()
    average_prices_by_neighborhood.reset_index(inplace=True)
    average_prices_by_neighborhood_plot = average_prices_by_neighborhood.hvplot.line(x='year',y="sale_price_sqr_foot",ylabel='Avg. Sale Price per Square Foot',        groupby='neighborhood')
    return average_prices_by_neighborhood_plot


def top_most_expensive_neighborhoods():
    """Top 10 Most Expensive Neighborhoods."""
    mean_neighborhood = sfo_data.groupby("neighborhood").mean()
    mean_neighborhood = mean_neighborhood.sort_values(by="sale_price_sqr_foot", ascending=False)
    mean_neighborhood_top_10 = mean_neighborhood.head(10)
    mean_neighborhood_top_10.reset_index(inplace=True)
    mean_neighborhood_top_10_plot = mean_neighborhood_top_10.hvplot.bar(x='neighborhood',y="sale_price_sqr_foot",ylabel='Avg. Sale Price per Square Foot',rot=90,width=800,height=600)
    return mean_neighborhood_top_10_plot


def parallel_coordinates():
    """Parallel Coordinates Plot."""
    
    mean_neighborhood = sfo_data.groupby("neighborhood").mean()
    mean_neighborhood = mean_neighborhood.sort_values(by="sale_price_sqr_foot", ascending=False)
    mean_neighborhood_top_10 = mean_neighborhood.head(10)
    mean_neighborhood_top_10.reset_index(inplace=True)
    mean_neighborhood_top_10_plot_parra_coor = px.parallel_coordinates(mean_neighborhood_top_10, color='sale_price_sqr_foot')
    return mean_neighborhood_top_10_plot_parra_coor


def parallel_categories():
    """Parallel Categories Plot."""
    
    mean_neighborhood = sfo_data.groupby("neighborhood").mean()
    mean_neighborhood = mean_neighborhood.sort_values(by="sale_price_sqr_foot", ascending=False)
    mean_neighborhood_top_10 = mean_neighborhood.head(10)
    mean_neighborhood_top_10.reset_index(inplace=True)
    mean_neighborhood_top_10_plot_parra_cat = px.parallel_coordinates(mean_neighborhood_top_10, color='sale_price_sqr_foot')
    return mean_neighborhood_top_10_plot_parra_cat


def neighborhood_map():
    """Neighborhood Map"""
    
    #cant get mapbox to work



## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [23]:
tab_1_columns = pn.Column(
    ("# San Francisco Housing Rental Analysis - 2010 to 2016"),
    ("## Housing Units Per Year"),
    housing_units_per_year,
    ("## Average Gross Rent Per Year"),
    average_gross_rent,
    ("## Average Sales Price Per Year"),
    average_sales_price
)

tab_2_columns = pn.Column(
    ("# San Francisco Neighborhood Analysis - 2010 to 2016"),
    ("## Average Prices by Neighborhood"),
    average_price_by_neighborhood,
    ("## Top 10 Most Expensive Neighborhoods"),
    top_most_expensive_neighborhoods
)

tab_3_columns = pn.Column(
        ("# Parallel Coordinates and Parallel Categories Analysis"),
    ("## Parallel Coordinates Plot"),
    parallel_coordinates,
    ("## Parallel Categories Plot"),
    parallel_categories
)



panel = pn.Tabs(
     ("Rental Analysis", tab_1_columns),
    ("Neighborhood Analysis", tab_2_columns),
    ("Parallel",tab_3_columns),
)

## Serve the Panel Dashboard

In [26]:
panel.servable()

Tabs
    [0] Column
        [0] Markdown(str)
        [1] Markdown(str)
        [2] Column
            [0] Column()
            [1] Row
                [0] Plotly(Figure, name='interactive01641')
        [3] Markdown(str)
        [4] Column
            [0] Column()
            [1] Row
                [0] Plotly(Figure, name='interactive01649')
        [5] Markdown(str)
        [6] Column
            [0] Column()
            [1] Row
                [0] Plotly(Figure, name='interactive01657')
    [1] Column
        [0] Markdown(str)
        [1] Markdown(str)
        [2] Column
            [0] Column()
            [1] Row
                [0] Row(name='interactive01668')
                    [0] HoloViews(DynamicMap, name='interactive01668')
                    [1] Column
                        [0] WidgetBox
                            [0] Select(margin=(20, 20, 20, 20), name='neighborhood', options=['Alamo Square', ...], value='Alamo Square', width=250)
                        [1] VSpacer()
        [3] Markdown(str)
        [4] Column
            [0] Column()
            [1] Row
                [0] HoloViews(Bars, name='interactive01708')
    [2] Column
        [0] Markdown(str)
        [1] Markdown(str)
        [2] Column
            [0] Column()
            [1] Row
                [0] Plotly(Figure, name='interactive01798')
        [3] Markdown(str)
        [4] Column
            [0] Column()
            [1] Row
                [0] Plotly(Figure, name='interactive01806')